In [1]:
from flask import Flask, render_template, redirect, url_for
from flask_pymongo import PyMongo
import scraping

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\hajaf\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\hajaf\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [2]:
app = Flask(__name__)


In [3]:
# Use flask_pymongo to set up mongo connection
app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"
mongo = PyMongo(app)

In [4]:

@app.route("/")
def index():
    mars = mongo.db.mars.find_one()
    return render_template("index.html", mars=mars)

In [5]:
@app.route("/scrape")
def scrape():
    mars = mongo.db.mars
    mars_data = scraping.scrape_all()
    mars.update_one({}, {"$set":mars_data}, upsert=True)
    return redirect('/', code=302)

In [6]:
mars.update_one({}, {"$set":mars_data}, upsert=True)
return redirect('/', code=302)

NameError: name 'mars' is not defined

In [ ]:
if __name__ == "__main__":
     app.run()

In [ ]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')
slide_elem.find('div', class_='content_title')

# Use the parent element to find the first 'a' tag and save it as 'news_title'
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p